In [1]:
# !pip install -qU langchain-community SQLAlchemy langchain-openai

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

#### storage를 사용하려면 다음 2가지만 제공하면 됩니다:

1. Session Id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자입니다.

2. Connection string - 데이터베이스 연결을 지정하는 문자열입니다. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달됩니다.

- SQLChatMessageHistory 클래스를 사용하여 대화 메시지 기록을 관리합니다.
- session_id와 connection_string을 인자로 전달하여 SQLChatMessageHistory 인스턴스를 생성합니다.
- add_user_message 메서드를 사용하여 사용자 메시지를 대화 기록에 추가합니다.
- add_ai_message 메서드를 사용하여 AI 메시지를 대화 기록에 추가합니다.

In [3]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달합니다.
chat_message_history = SQLChatMessageHistory(
    session_id="sql_chat_history", connection_string="sqlite:///sqlite.db"
)

# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "Hi! My name is Teddy. I am a AI programmer. Nice to meet you!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("Hi Teddy! Nice to meet you too!")

c:\Users\UserK\miniforge3\envs\rag\lib\site-packages\langchain_community\chat_message_histories\sql.py:186: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. Use Use connection instead instead.
  warn_deprecated(


In [4]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='Hi! My name is Teddy. I am a AI programmer. Nice to meet you!'),
 AIMessage(content='Hi Teddy! Nice to meet you too!')]

### Chain 에 적용

In [5]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

- ChatPromptTemplate을 사용하여 프롬프트 템플릿을 정의합니다.
    - "system" 메시지: "You are a helpful assistant."
    - MessagesPlaceholder를 사용하여 "history" 변수를 프롬프트에 포함시킵니다.
    - "human" 메시지: "{question}"
- 프롬프트 템플릿과 ChatOpenAI 모델을 파이프라인으로 연결하여 chain을 생성합니다.

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메시지를 설정하여 어시스턴트의 역할을 정의합니다.
        ("system", "You are a helpful assistant."),
        # 이전 대화 내용을 포함하기 위한 플레이스홀더를 추가합니다.
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),  # 사용자의 질문을 입력받는 메시지를 설정합니다.
    ]
)

chain = (
    prompt | ChatOpenAI()
)  # 프롬프트와 ChatOpenAI 모델을 연결하여 체인을 생성합니다.

- RunnableWithMessageHistory 클래스를 사용하여 chain과 메시지 기록을 연결합니다.
- lambda 함수를 사용하여 session_id에 따라 SQLChatMessageHistory 인스턴스를 생성합니다.
    - SQLChatMessageHistory는 SQLite 데이터베이스(sqlite.db)에 연결되어 채팅 메시지 기록을 저장합니다.
- input_messages_key를 "question"으로 설정하여 입력 메시지의 키를 지정합니다.
- history_messages_key를 "history"로 설정하여 기록 메시지의 키를 지정합니다.

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),  # session_id를 기반으로 SQLChatMessageHistory 객체를 생성하는 람다 함수
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="history",  # 대화 기록 메시지의 키를 "history"로 설정
)

In [8]:
# 세션 ID를 구성하는 곳입니다.
config = {"configurable": {"session_id": "sql_chat_history"}}

In [9]:
# 질문 "Whats my name"과 설정을 사용하여 대화 기록이 있는 체인을 호출합니다.
chain_with_history.invoke({"question": "Whats my name?"}, config=config)

AIMessage(content='Your name is Teddy.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 56, 'total_tokens': 61}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e67be696-fdff-41db-82d9-39c9808c93dc-0', usage_metadata={'input_tokens': 56, 'output_tokens': 5, 'total_tokens': 61})